In [25]:
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mutual_info_score
from xgboost import XGBClassifier
print("User Current Version:-", sys.version)

User Current Version:- 3.8.15 (default, Nov 24 2022, 09:04:07) 
[Clang 14.0.6 ]


In [2]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client.from_service_account_json("../config/sacred-garden-369506-870f85c5921d.json")

query = """
    SELECT *
    FROM `sacred-garden-369506.personal_project.telco-customer_churn`
"""
query_job = (
    client
    .query(query).to_dataframe()
    .query("TotalCharges!=' '")
    .assign(TotalCharges = lambda x:x.TotalCharges.astype(float))
    .assign(SeniorCitizen = lambda x:x.SeniorCitizen.astype(str))
    .reset_index(drop=True)
)

df = query_job.drop('customerID',axis=1)


In [3]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,True,False,49,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.70,False
1,Female,0,True,True,6,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.50,False
2,Female,0,True,True,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
3,Male,0,True,False,36,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4,Female,0,False,False,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.30,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   SeniorCitizen     7032 non-null   object 
 2   Partner           7032 non-null   boolean
 3   Dependents        7032 non-null   boolean
 4   tenure            7032 non-null   Int64  
 5   PhoneService      7032 non-null   boolean
 6   MultipleLines     7032 non-null   object 
 7   InternetService   7032 non-null   object 
 8   OnlineSecurity    7032 non-null   object 
 9   OnlineBackup      7032 non-null   object 
 10  DeviceProtection  7032 non-null   object 
 11  TechSupport       7032 non-null   object 
 12  StreamingTV       7032 non-null   object 
 13  StreamingMovies   7032 non-null   object 
 14  Contract          7032 non-null   object 
 15  PaperlessBilling  7032 non-null   boolean
 16  PaymentMethod     7032 non-null   object 


In [5]:
cat_list = df.select_dtypes(exclude=['float64','Int64']).columns.tolist()
cat_list = [e for e in cat_list if e not in ('customerID', 'Churn')]

In [6]:
mutual_info_dict = {}
for i in cat_list:
    mutual_info_dict[i] = mutual_info_score(df[i],df.Churn)

In [8]:
dict(sorted(mutual_info_dict.items(), key=lambda item: item[1],reverse=True))

{'Contract': 0.09818219921734034,
 'OnlineSecurity': 0.06452824152301159,
 'TechSupport': 0.06287279751958623,
 'InternetService': 0.055393693585615335,
 'OnlineBackup': 0.0466589944654092,
 'PaymentMethod': 0.044422924978715875,
 'DeviceProtection': 0.04378428627008563,
 'StreamingMovies': 0.03191837098237602,
 'StreamingTV': 0.03180253675235035,
 'PaperlessBilling': 0.019119265234473853,
 'Dependents': 0.014270326012253198,
 'Partner': 0.011383147791626885,
 'SeniorCitizen': 0.01053262470911398,
 'MultipleLines': 0.0007983565971204365,
 'PhoneService': 6.914412223160693e-05,
 'gender': 3.6504338027820715e-05}

In [14]:
cat_list

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [15]:
gender_encoded = pd.DataFrame(pd.get_dummies(df['gender'],prefix='gender',prefix_sep=':'))

partner_encoded = pd.DataFrame(pd.get_dummies(df['Partner'],prefix='Partner',prefix_sep=':'))

dependents_encoded = pd.DataFrame(pd.get_dummies(df['Dependents'],prefix='Dependents',prefix_sep=':'))

phone_service_encoded = pd.DataFrame(pd.get_dummies(df['PhoneService'],prefix='PhoneService',prefix_sep=':'))

multiple_lines_encoded = pd.DataFrame(pd.get_dummies(df['MultipleLines'],prefix='MultipleLines',prefix_sep=':'))

internet_service_encoded = pd.DataFrame(pd.get_dummies(df['InternetService'],prefix='InternetService',prefix_sep=':'))

online_security_encoded = pd.DataFrame(pd.get_dummies(df['OnlineSecurity'],prefix='OnlineSecurity',prefix_sep=':'))

online_backup_encoded = pd.DataFrame(pd.get_dummies(df['OnlineBackup'],prefix='OnlineBackup',prefix_sep=':'))

device_protection_encoded = pd.DataFrame(pd.get_dummies(df['DeviceProtection'],prefix='DeviceProtection',prefix_sep=':'))

tech_support_encoded = pd.DataFrame(pd.get_dummies(df['TechSupport'],prefix='TechSupport',prefix_sep=':'))

streaming_tv_encoded = pd.DataFrame(pd.get_dummies(df['StreamingTV'],prefix='StreamingTV',prefix_sep=':'))

streaming_movie_encoded = pd.DataFrame(pd.get_dummies(df['StreamingMovies'],prefix='StreamingMovies',prefix_sep=':'))

contract_encoded = pd.DataFrame(pd.get_dummies(df['Contract'],prefix='Contract',prefix_sep=':'))

paperless_billing_encoded = pd.DataFrame(pd.get_dummies(df['PaperlessBilling'],prefix='PaperlessBilling',prefix_sep=':'))

payment_method_encoded = pd.DataFrame(pd.get_dummies(df['PaymentMethod'],prefix='PaymentMethod',prefix_sep=':'))

In [19]:
df_final = pd.concat([gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,multiple_lines_encoded,
        internet_service_encoded,online_security_encoded,online_backup_encoded,device_protection_encoded,
        tech_support_encoded,streaming_tv_encoded,streaming_movie_encoded,contract_encoded,
        paperless_billing_encoded,payment_method_encoded,df['tenure'],df['SeniorCitizen'].astype(int),df['MonthlyCharges'],
        df['TotalCharges'],df['Churn']],axis=1)

In [22]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_final.Churn)
df_final.Churn = le.transform(df_final.Churn)


In [24]:
from sklearn.model_selection import train_test_split
X,y = df_final.drop('Churn',axis=1),df_final['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
xgb_cl = XGBClassifier()
xgb_cl.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [27]:
y_pred = xgb_cl.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[936, 111],
       [166, 194]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8031272210376688